In [1]:
import langchain

In [4]:
# !pip -q install langchain openai tiktoken chromadb python-dotenv

In [52]:
# !pip install PyMuPDF

In [6]:
import os
from dotenv import load_dotenv

load_dotenv()
# os.environ["OPENAI_API_KEY"] = ""

True

In [53]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA
from langchain.document_loaders import PyMuPDFLoader
from langchain.document_loaders import DirectoryLoader

In [49]:
# Load and process the text files
# loader = TextLoader('single_text_file.txt')
loader = DirectoryLoader('./data/', glob="./*.pdf", loader_cls=PyMuPDFLoader)

documents = loader.load()

In [50]:
#splitting the text into
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(documents)

In [109]:
## Embedding

In [ ]:
## here we are using OpenAI embeddings but in future we will swap out to local embeddings
embedding = OpenAIEmbeddings()

In [ ]:
from langchain.embeddings import HuggingFaceInstructEmbeddings

## Here is the nmew embeddings being use
embedding = HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-xl",
                                          model_kwargs={"device": "cuda"})

## Create a Vector DB

In [54]:
# Embed and store the texts
# Supplying a persist_directory will store the embeddings on disk
persist_directory = 'db'

vectordb = Chroma.from_documents(documents=texts, 
                                 embedding=embedding,
                                 persist_directory=persist_directory)

In [55]:
# persiste the db to disk
vectordb.persist()
vectordb = None

In [56]:
# Now we can load the persisted database from disk, and use it as normal. 
vectordb = Chroma(persist_directory=persist_directory, 
                  embedding_function=embedding)

## Make a retriever

In [86]:
# retriever = vectordb.as_retriever()
retriever = vectordb.as_retriever(search_kwargs={"k": 2})
# retriever.search_type
# retriever.search_kwargs

In [87]:
docs = retriever.get_relevant_documents("과거는 반복된다")

In [88]:
len(docs)

2

In [89]:
docs

[Document(page_content='인 차원으로 풀이되어 옴에 따라 읍면동과 같은 소지역 단위에서의 빈집정비 방향\n설정에 유효한 설명이 부족하였음. 이상의 분석결과는 같은 지자체 안에서도 빈집\n발생 정도에 차이가 나타나는 이유를 설명하는 틀을 제공하며, 빈집 발생을 완화하\n기 위해 취할 수 있는 정책적 방안 모색을 가능케 함\n  ◦ 빈집 발생 및 증가 현상에 대한 이해 증대\n   - 분석결과 읍면동 내 빈집 발생 및 증가 현상은 인구구조 특성뿐만 아니라 지역의\n경제적 상황, 생활편의 등 지역생태학적 요인이 복합적으로 작용한 결과임. 단순히\n중앙 및 지방자치단체기관이 지방으로 이전되었다고 해서 빈집 발생이 억제되는 효\n과로 이어지지 않으며, 분석결과에서도 자치단체기관수가 빈집 발생에 유의한 영향\n을 미치지 않는 것으로 나타남. 또한 도시 지역의 빈집 발생은 폐업 사업체수 등\n지역의 경제적 상황과 밀접히 관련되어 있다는 분석결과는 최근 대기업 두 곳이 연\n이어 철수한 군산 등 전북 지역의 빈집 발생 심화현상에 적확한 설명을 제공함', metadata={'source': 'data/지역의 사회구조적 특성이 빈집 형성에 미친 영향에 관한 패널 분석(우수).pdf', 'file_path': 'data/지역의 사회구조적 특성이 빈집 형성에 미친 영향에 관한 패널 분석(우수).pdf', 'page': 4, 'total_pages': 6, 'format': 'PDF 1.4', 'title': '', 'author': 'mhan', 'subject': '', 'keywords': '', 'creator': 'Hwp 2018 10.0.0.9139', 'producer': 'Hancom PDF 1.3.0.538', 'creationDate': "D:20200724181319+09'00'", 'modDate': "D:20200724181319+09'00'", 'trapped': ''}),
 Document(page_content='일수록 유의하게 빈집이 

## Make a Chin

In [105]:
# create the chain to answer questions 
qa_chain = RetrievalQA.from_chain_type(llm=OpenAI(), 
                                  chain_type="stuff", 
                                  retriever=retriever, 
                                  return_source_documents=True)

In [101]:
print(qa_chain.combine_documents_chain.llm_chain.prompt.template)

Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

{context}

Question: {question}
Helpful Answer:


In [ ]:
## Cite sources

import textwrap

def wrap_text_preserve_newlines(text, width=110):
    # Split the input text into lines based on newline characters
    lines = text.split('\n')

    # Wrap each line individually
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]

    # Join the wrapped lines back together using newline characters
    wrapped_text = '\n'.join(wrapped_lines)

    return wrapped_text

In [106]:
## Cite sources
def process_llm_response(llm_response):
    print(llm_response['result'])
    # print(wrap_text_preserve_newlines(llm_response['result']))
    print('\n\nSources:')
    for source in llm_response["source_documents"]:
        print(source.metadata['source'])

In [107]:
# full example
query = "사회구조적 특성이 빈집 형성에 끼친 영항에 대해 설명해주세요"
llm_response = qa_chain(query)
process_llm_response(llm_response)

 
사회구조적 특성이 빈집 형성에 끼친 영향은 다양합니다. 예를 들어, 사회구조가 공동체 중심적이라면, 가구가 생기거나 사람들이 이사하는 경우 빈집이 생기기가 쉽습니다. 반면, 사회구조가 개인 중심적이라면, 빈집이


Sources:


## Deleteing the DB

In [97]:
# To cleanup, you can delete the collection
vectordb.delete_collection()
vectordb.persist()

# delete the directory
!rm -rf db/

## Starting again loading the db

In [ ]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings

from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA

In [99]:
persist_directory = 'db'
embedding = OpenAIEmbeddings()

vectordb = Chroma(persist_directory=persist_directory, 
                  embedding_function=embedding,
                   )

retriever = vectordb.as_retriever(search_kwargs={"k": 2})

In [ ]:
# Set up the turbo LLM
turbo_llm = ChatOpenAI(
    temperature=0,
    model_name='gpt-3.5-turbo'
)

In [ ]:
# create the chain to answer questions 
qa_chain = RetrievalQA.from_chain_type(llm=turbo_llm, 
                                  chain_type="stuff", 
                                  retriever=retriever, 
                                  return_source_documents=True)

In [ ]:
print(qa_chain.combine_documents_chain.llm_chain.prompt.messages[0].prompt.template)

In [ ]:
print(qa_chain.combine_documents_chain.llm_chain.prompt.messages[1].prompt.template)

In [ ]:
## Cite sources
def process_llm_response(llm_response):
    print(llm_response['result'])
    print('\n\nSources:')
    for source in llm_response["source_documents"]:
        print(source.metadata['source'])